In [1]:
import sqlite3
import datetime as dt

In [2]:
# Check the API version.
sqlite3.version_info

(2, 6, 0)

In [3]:
# Check the underlying SQLite version.
sqlite3.sqlite_version

'3.45.3'

In [4]:
# Connect to an in-memory database.
db = sqlite3.connect(":memory:")
db

In [5]:
# Create a Cursor object.
cursor = db.cursor()
cursor

In [6]:
# Attributes from the Cursor.
# - execute methods -> Perform SQL statements.
# - fetch methods   -> Fetch data from a SELECT resultset.
# - description     -> Metadata for a SELECT resultset.
# - rowcount        -> Rows affected by the last SQL statement.
for attribute in dir(cursor):
    if attribute.startswith("_"):
        continue
    print(attribute)

arraysize
close
connection
description
execute
executemany
executescript
fetchall
fetchmany
fetchone
lastrowid
row_factory
rowcount
setinputsizes
setoutputsize


## Create table and data

In [7]:
cursor.execute("DROP TABLE IF EXISTS Klanten;")

In [8]:
# Create a customer table.
cursor.execute(
    """
    CREATE TABLE Klanten (
        KlantId INTEGER,
        Voornaam TEXT,
        Achternaam TEXT NOT NULL,
        GeboorteDatum DATE,
        PRIMARY KEY(KlantId)
    );
    """
)

In [9]:
# Insert two new customers.
cursor.execute(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES
        ('Ingrid', 'Jansen', '1984-09-06'),
        ('Henk', 'Knol', '1957-08-23')
    ;
    """
)

In [10]:
# Verify the insert statement worked...
cursor.rowcount

2

## Select data

In [11]:
# Perform a SELECT query.
cursor.execute("SELECT * FROM Klanten")

In [12]:
# No rowcount
cursor.rowcount

-1

In [13]:
# Look at the metadata
# - name
# - type_code
# - display_size
# - internal_size
# - precision
# - scale
# - null_ok
cursor.description

(('KlantId', None, None, None, None, None, None),
 ('Voornaam', None, None, None, None, None, None),
 ('Achternaam', None, None, None, None, None, None),
 ('GeboorteDatum', None, None, None, None, None, None))

In [14]:
# Extract column names from the description.
column_names = [field_meta[0] for field_meta in cursor.description]
column_names

['KlantId', 'Voornaam', 'Achternaam', 'GeboorteDatum']

In [15]:
# Use fetch methods to get the data.
row = cursor.fetchone()
row

(1, 'Ingrid', 'Jansen', '1984-09-06')

In [16]:
# Row is a simple tuple.
type(row)

tuple

In [17]:
# Convert to dict.
dict(zip(column_names, row))

{'KlantId': 1,
 'Voornaam': 'Ingrid',
 'Achternaam': 'Jansen',
 'GeboorteDatum': '1984-09-06'}

## Row Factory

In [18]:
# Set a row factory.
db.row_factory = sqlite3.Row

In [19]:
# Create a new cursor using the row factory.
cursor = db.cursor()

In [20]:
# Perform a query and fetch the first row.
cursor.execute("SELECT * FROM Klanten;")
row = cursor.fetchone()

In [21]:
# Note that result is now a Row object.
type(row)

sqlite3.Row

In [22]:
# Get the column names from the Row object.
row.keys()

['KlantId', 'Voornaam', 'Achternaam', 'GeboorteDatum']

In [23]:
# Index by column name.
row["Voornaam"]

'Ingrid'

In [24]:
# Or by index.
row[1]

'Ingrid'

### Insert data from Python

In [26]:
# Customer data as dict.
customer = {
    "Voornaam": "Maria",
    "Achternaam": "Klomp",
    "GeboorteDatum": dt.date(1990, 5, 7)
}


In [27]:
# Define query as a string template...
template = """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (
        '{Voornaam}', '{Achternaam}', '{GeboorteDatum}'
    );
"""

In [28]:
# Insert values using Python formatting...
print(template.format(**customer))


    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (
        'Maria', 'Klomp', '1990-05-07'
    );



In [29]:
# Customer data with missing value.
customer = {
    "Voornaam": "Maria",
    "Achternaam": "Klomp",
    "GeboorteDatum": None,
}

In [30]:
# Bad idea...
print(template.format(**customer))


    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (
        'Maria', 'Klomp', 'None'
    );



In [34]:
# Customer selection query template.
template = "SELECT * FROM Klanten WHERE Achternaam = '{lastname}'"

In [35]:
# Craft a condition that is always true...
lastname = "x' OR 1 = 1 OR 'x"
formatted = template.format(lastname=lastname)
formatted


"SELECT * FROM Klanten WHERE Achternaam = 'x' OR 1 = 1 OR 'x'"

In [41]:
# Query returns all our customer data!
for row in cursor.execute(formatted).fetchall():
    print(dict(row))

{'KlantId': 1, 'Voornaam': 'Ingrid', 'Achternaam': 'Jansen', 'GeboorteDatum': '1984-09-06'}
{'KlantId': 2, 'Voornaam': 'Henk', 'Achternaam': 'Knol', 'GeboorteDatum': '1957-08-23'}


In [ ]:
# Good idea: Use a parametrized query.
cursor.execute(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (:Voornaam, :Achternaam, :GeboorteDatum);
    """,
    customer
)
cursor.rowcount

In [ ]:
# Using positional parameters.
cursor.execute(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (?, ?, ?);
    """,
    ("Piet", "Klaassen", dt.date(1978, 9, 4))
)
cursor.rowcount

In [ ]:
# List of customer records.
customers = [
    {"Voornaam": "Noah", "Achternaam": "Versteeg", "GeboorteDatum": None},
    {"Voornaam": "Mark", "Achternaam": "Vos", "GeboorteDatum": dt.date(1998, 1, 4)},
    {"Voornaam": "Bart", "Achternaam": "Poot", "GeboorteDatum": dt.date(1955, 8, 21)},
]

In [ ]:
# Insert all customers in one go!
cursor.executemany(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (:Voornaam, :Achternaam, :GeboorteDatum);
    """,
    customers
)
cursor.rowcount